In [42]:
import pandas as pd  # For handling structured data
import geopandas as gpd  # For handling geospatial data
import numpy as np  # For numerical computations
import shapely  # For handling geometric objects
import requests  # For making HTTP requests
from arcgis.features import FeatureLayer  # For working with ArcGIS feature layers
from sklearn.ensemble import RandomForestRegressor  # ML model for regression
from sklearn.model_selection import train_test_split  # Splitting data into training/testing sets
from sklearn.metrics import mean_absolute_error, r2_score  # Model evaluation metrics
from datetime import datetime, timezone
import openmeteo_requests
from retry_requests import retry
import requests_cache


In [ ]:

### Load Odor Complaints Data from San Diego County GIS ###
# The data source is an ArcGIS REST API providing odor complaint reports in San Diego County.

complaints_url = "https://gis-public.sandiegocounty.gov/arcgis/rest/services/Hosted/SDAPCD_Complaints/FeatureServer/0/query"
params = {"where": "1=1", "outFields": "*", "f": "geojson"}

# Request the data from the API
response = requests.get(complaints_url, params=params)

# Load the response into a GeoDataFrame
complaints_gdf = gpd.read_file(response.json())

# Ensure geospatial data is correctly formatted
complaints_gdf = complaints_gdf.set_geometry(gpd.points_from_xy(complaints_gdf["longitude"], complaints_gdf["latitude"]))
complaints_gdf.head()

In [73]:

### Load Wind Data from Open-Meteo API ###
# Open-Meteo provides historical and real-time weather data, including wind speed and direction.

def get_wind_data(lat, lon, start_date, end_date):
    """
    Fetches historical wind data for a given location and time range from Open-Meteo.
    """
    open_meteo_url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": ["wind_speed_10m", "wind_direction_10m", "temperature_2m", "relative_humidity_2m","wind_gusts_10m"],
        "timezone": "America/Los_Angeles"
    }
    
    response = requests.get(open_meteo_url, params=params)
    if response.status_code == 200:
        wind_data = response.json()
        df = pd.DataFrame({
            "time": wind_data["hourly"]["time"],
            "wind_speed": wind_data["hourly"]["wind_speed_10m"],
            "wind_dir": wind_data["hourly"]["wind_direction_10m"],
            "temperature": wind_data["hourly"]["temperature_2m"],
            "humidity": wind_data["hourly"]["relative_humidity_2m"],
            "wind_gusts": wind_data["hourly"]["wind_gusts_10m"],
            
        })
        df["time"] = pd.to_datetime(df["time"])
        df["time"]=df["time"].dt.tz_localize("America/Los_Angeles")
        return df
    else:
        raise Exception(f"Failed to fetch wind data from Open-Meteo. {response.status_code} response:{response.text} ")

# Example: Load wind data for a specific location and date range
wind_df = get_wind_data(lat=32.7157, lon=-117.1611, start_date="2025-03-01", end_date="2025-03-04")
wind_df = wind_df.set_index(pd.DatetimeIndex(wind_df['time']))
wind_df.drop('time', inplace=True, axis=1)
wind_df.head()

,wind_speed,wind_dir,temperature,humidity,wind_gusts
time,,,,,
2025-03-01 00:00:00-08:00,2.9,248.0,12.2,94.0,6.8
2025-03-01 01:00:00-08:00,2.7,247.0,12.3,94.0,8.6
2025-03-01 02:00:00-08:00,1.4,270.0,12.9,91.0,7.2
2025-03-01 03:00:00-08:00,0.9,281.0,12.5,93.0,3.6
2025-03-01 04:00:00-08:00,1.9,202.0,11.7,95.0,4.0


In [74]:


### Load H₂S Sensor Data ###
# The H₂S sensor data measures hydrogen sulfide levels over time.
hs2url= 'https://oss.resilientservice.mooo.com/test/sd_apcd_air/output/h2s.csv'
h2s_sensor_data_all = pd.read_csv(hs2url)  # Columns: time, H2S_level

h2s_sensor_data_all["time"] = pd.to_datetime(h2s_sensor_data_all["Date with time"])  # Ensure time format consistency
h2s_sensor_data_all["time"]=h2s_sensor_data_all["time"].dt.tz_localize("America/Los_Angeles", ambiguous=True)
h2s_sensor_data=h2s_sensor_data_all[h2s_sensor_data_all['Site Name']=='NESTOR - BES']

In [75]:
h2s_sensor_data = h2s_sensor_data.set_index(pd.DatetimeIndex(h2s_sensor_data['time']))
h2s_sensor_data.drop('time', inplace=True, axis=1)
h2s_sensor_data

,Parameter,Site Name,Date with time,Result,Qualifier,Original Value,level
time,,,,,,,
2025-03-03 00:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-03T00:00:00,0.7,NaN,.7,green
2025-03-03 01:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-03T01:00:00,0.8,NaN,.8,green
2025-03-03 02:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-03T02:00:00,0.8,NaN,.8,green
2025-03-03 03:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-03T03:00:00,0.8,NaN,.8,green
2025-03-03 04:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-03T04:00:00,0.8,NaN,.8,green
...,...,...,...,...,...,...,...
2025-03-04 07:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-04T07:00:00,2.5,NaN,2.5,green
2025-03-04 08:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-04T08:00:00,2.4,NaN,2.4,green
2025-03-04 09:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-04T09:00:00,1.6,NaN,1.6,green


In [82]:

# Merge wind and H₂S sensor data on time
merged_df = pd.merge(h2s_sensor_data, wind_df, on="time", how="inner")
merged_df.dropna(subset=['Result', "wind_speed", "wind_dir", "temperature", "humidity", "wind_gusts"],inplace=True)
merged_df



,Parameter,Site Name,Date with time,Result,Qualifier,Original Value,level,wind_speed,wind_dir,temperature,humidity,wind_gusts
time,,,,,,,,,,,,
2025-03-02 00:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-02T00:00:00,17.4,NaN,17.4,yellow,6.1,146.0,12.7,83.0,15.5
2025-03-02 01:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-02T01:00:00,10.8,NaN,10.8,yellow,7.8,157.0,13.0,80.0,18.4
2025-03-02 02:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-02T02:00:00,4.1,NaN,4.1,green,8.6,168.0,13.1,81.0,20.2
2025-03-02 03:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-02T03:00:00,7.9,NaN,7.9,yellow,6.6,201.0,13.1,83.0,20.5
2025-03-02 04:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-02T04:00:00,16.4,NaN,16.4,yellow,6.0,224.0,13.1,85.0,15.8
2025-03-02 05:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-02T05:00:00,18.4,NaN,18.4,yellow,4.2,211.0,13.5,76.0,12.2
2025-03-02 06:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-02T06:00:00,19.7,NaN,19.7,yellow,4.5,178.0,13.1,77.0,10.1
2025-03-02 07:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-02T07:00:00,10.0,NaN,10.0,yellow,5.8,216.0,13.0,80.0,11.2
2025-03-02 08:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-02T08:00:00,1.9,NaN,1.9,green,7.1,223.0,14.3,73.0,19.8


In [93]:


### Gaussian Plume Model ###
# A more detailed implementation of the **Gaussian Plume Model** for estimating pollutant dispersion.
# This model assumes steady-state conditions and calculates the concentration of pollutants downwind.

def gaussian_plume(x, y, z, Q, u, sigma_y, sigma_z, H):
    """
    Computes the Gaussian Plume Model concentration at (x, y, z).
    
    Parameters:
    - x, y: Downwind (x) and crosswind (y) distances (meters)
    - z: Height above the ground (meters)
    - Q: Source emission rate (g/s)
    - u: Wind speed at stack height (m/s)
    - sigma_y: Horizontal dispersion coefficient (meters)
    - sigma_z: Vertical dispersion coefficient (meters)
    - H: Effective stack height (meters)

    Returns:
    - C: Pollutant concentration at (x, y, z) in g/m³
    """
    C = (Q / (2 * np.pi * u * sigma_y * sigma_z)) * np.exp(-y**2 / (2 * sigma_y**2)) * (
        np.exp(- (z - H) ** 2 / (2 * sigma_z ** 2)) + np.exp(- (z + H) ** 2 / (2 * sigma_z ** 2))
    )
    return C

# Assume a fixed emission source located at (x=0, y=0, H=10m), with Q=10 g/s emission rate.
merged_df["plume_estimate"] = gaussian_plume(
    x=merged_df["wind_speed"],  # Approximating downwind distance using wind speed as a proxy
    y=0,  # Assuming along-wind estimation
    z=1.5,  # Human breathing height
    Q=100,  # Example emission rate in g/s
    u=merged_df["wind_speed"],
    sigma_y=50,  # Empirical dispersion coefficient
    sigma_z=20,  # Empirical dispersion coefficient
    H=10  # Assumed source stack height
)
merged_df["plume_estimate_2"] = gaussian_plume(
    x=merged_df["wind_speed"],  # Approximating downwind distance using wind speed as a proxy
    y=0,  # Assuming along-wind estimation
    z=1.5,  # Human breathing height
    Q=1000,  # Example emission rate in g/s
    u=merged_df["wind_speed"],
    #sigma_y=50,  # Empirical dispersion coefficient
    sigma_y=100,  # Empirical dispersion coefficient
    sigma_z=20,  # Empirical dispersion coefficient
    H=10  # Assumed source stack height
)
merged_df["plume_estimate_3"] = gaussian_plume(
    x=merged_df["wind_speed"],  # Approximating downwind distance using wind speed as a proxy
    y=0,  # Assuming along-wind estimation
    z=1.5,  # Human breathing height
    Q=10000,  # Example emission rate in g/s
    u=merged_df["wind_speed"],
    #sigma_y=50,  # Empirical dispersion coefficient
    sigma_y=1,  # Empirical dispersion coefficient
    sigma_z=50,  # Empirical dispersion coefficient
    H=10  # Assumed source stack height
)

In [94]:
merged_df

,Parameter,Site Name,Date with time,Result,Qualifier,Original Value,level,wind_speed,wind_dir,temperature,humidity,wind_gusts,plume_estimate,plume_estimate_10,plume_estimate_2,plume_estimate_3
time,,,,,,,,,,,,,,,,
2025-03-02 00:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-02T00:00:00,17.4,NaN,17.4,yellow,6.1,146.0,12.7,83.0,15.5,0.004595,0.002298,0.022977,10.225317
2025-03-02 01:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-02T01:00:00,10.8,NaN,10.8,yellow,7.8,157.0,13.0,80.0,18.4,0.003594,0.001797,0.017969,7.996722
2025-03-02 02:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-02T02:00:00,4.1,NaN,4.1,green,8.6,168.0,13.1,81.0,20.2,0.003259,0.001630,0.016297,7.252841
2025-03-02 03:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-02T03:00:00,7.9,NaN,7.9,yellow,6.6,201.0,13.1,83.0,20.5,0.004247,0.002124,0.021236,9.450672
2025-03-02 04:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-02T04:00:00,16.4,NaN,16.4,yellow,6.0,224.0,13.1,85.0,15.8,0.004672,0.002336,0.023360,10.395739
2025-03-02 05:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-02T05:00:00,18.4,NaN,18.4,yellow,4.2,211.0,13.5,76.0,12.2,0.006674,0.003337,0.033371,14.851056
2025-03-02 06:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-02T06:00:00,19.7,NaN,19.7,yellow,4.5,178.0,13.1,77.0,10.1,0.006229,0.003115,0.031146,13.860985
2025-03-02 07:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-02T07:00:00,10.0,NaN,10.0,yellow,5.8,216.0,13.0,80.0,11.2,0.004833,0.002417,0.024165,10.754213
2025-03-02 08:00:00-08:00,07 H2S PPB,NESTOR - BES,2025-03-02T08:00:00,1.9,NaN,1.9,green,7.1,223.0,14.3,73.0,19.8,0.003948,0.001974,0.019741,8.785132


In [96]:


### Machine Learning Model: Predicting H₂S Levels ###
# Selecting features for the model
X = merged_df[["plume_estimate_3", "wind_speed", "wind_dir", "temperature", "humidity", "wind_gusts"]]
#y = merged_df["H2S_level"]
y = merged_df["Result"]
# Splitting the dataset into training (80%) and testing (20%) subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a RandomForest regression model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model's performance
print("MAE:", mean_absolute_error(y_test, y_pred))  # Mean Absolute Error
print("R² Score:", r2_score(y_test, y_pred))  # R² Score to measure explained variance


MAE: 17.473666666666674
R² Score: 0.20406921855555193
